In [30]:
import numpy as np
from time import perf_counter

In [31]:
def mse(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return np.mean((y_true - y_pred) ** 2)


In [32]:
class LinearRegressionDot:

    def __init__(self, lr=0.01, n_iters=1000, fit_intercept=True):
        self.lr = float(lr)
        self.n_iters = int(n_iters)
        self.fit_intercept = bool(fit_intercept)
        self.theta_ = None  # includes intercept if fit_intercept=True

    def _prepare_X(self, X):
        X = np.asarray(X, dtype=float)
        if X.ndim == 1:
            X = X.reshape(-1, 1)
        if self.fit_intercept:
            return np.c_[np.ones(X.shape[0]), X]  # add bias column
        return X


    def intercept_(self):
        if self.theta_ is None:
            return None
        return float(self.theta_[0]) if self.fit_intercept else 0.0


    def coef_(self):
        if self.theta_ is None:
            return None
        return self.theta_[1:].copy() if self.fit_intercept else self.theta_.copy()

    def fit(self, X, y):
        Xb = self._prepare_X(X)
        y = np.asarray(y, dtype=float).reshape(-1)

        m, n = Xb.shape
        self.theta_ = np.zeros(n, dtype=float)

        for _ in range(self.n_iters):
            y_pred = np.dot(Xb, self.theta_)
            err = y_pred - y
            grad = np.dot(Xb.T, err) / m
            self.theta_ -= self.lr * grad

        return self

    def predict(self, X):
        Xb = self._prepare_X(X)
        return np.dot(Xb, self.theta_)




In [33]:
class LinearRegressionNoDot:

    def __init__(self, lr=0.01, n_iters=1000, fit_intercept=True):
        self.lr = float(lr)
        self.n_iters = int(n_iters)
        self.fit_intercept = bool(fit_intercept)
        self.theta_ = None  # includes intercept if fit_intercept=True


    def intercept_(self):
        if self.theta_ is None:
            return None
        return float(self.theta_[0]) if self.fit_intercept else 0.0


    def coef_(self):
        if self.theta_ is None:
            return None
        return self.theta_[1:].copy() if self.fit_intercept else self.theta_.copy()

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        if X.ndim == 1:
            X = X.reshape(-1, 1)
        y = np.asarray(y, dtype=float).reshape(-1)

        m, n = X.shape
        p = n + (1 if self.fit_intercept else 0)  # number of parameters
        self.theta_ = np.zeros(p, dtype=float)

        base = 1 if self.fit_intercept else 0

        for _ in range(self.n_iters):
            # 1) predictions (loop)
            y_pred = np.zeros(m, dtype=float)
            for i in range(m):
                s = self.theta_[0] if self.fit_intercept else 0.0
                for j in range(n):
                    s += self.theta_[base + j] * X[i, j]
                y_pred[i] = s

            #  calculating the gradient descent here
            grad = np.zeros(p, dtype=float)
            for i in range(m):
                diff = y_pred[i] - y[i]
                if self.fit_intercept:
                    grad[0] += diff
                for j in range(n):
                    grad[base + j] += diff * X[i, j]


            for k in range(p):
                grad[k] /= m
                self.theta_[k] -= self.lr * grad[k]

        return self

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        if X.ndim == 1:
            X = X.reshape(-1, 1)

        m, n = X.shape
        y_pred = np.zeros(m, dtype=float)

        base = 1 if self.fit_intercept else 0
        for i in range(m):
            s = self.theta_[0] if self.fit_intercept else 0.0
            for j in range(n):
                s += self.theta_[base + j] * X[i, j]  # theta is weight
            y_pred[i] = s

        return y_pred



In [34]:
def benchmark():
    np.random.seed(0) # randomly taking values
    m, n = 5000, 50
    X = np.random.randn(m, n)
    w_true = np.random.randn(n)
    y = X @ w_true + 0.5 * np.random.randn(m)

    lr = 0.01
    n_iters = 50

    dot_model = LinearRegressionDot(lr=lr, n_iters=n_iters, fit_intercept=True)
    nodot_model = LinearRegressionNoDot(lr=lr, n_iters=n_iters, fit_intercept=True)

    t0 = perf_counter()# 1st time is noted here
    dot_model.fit(X, y)
    t_dot = perf_counter() - t0 # differece in model trained here

    t0 = perf_counter()
    nodot_model.fit(X, y)
    t_nodot = perf_counter() - t0

    yhat_dot = dot_model.predict(X)
    yhat_nodot = nodot_model.predict(X)

    print(f"Vectorized (np.dot)   time: {t_dot:.4f} s | MSE: {mse(y, yhat_dot):.6f}")
    print(f"No dot (loops)        time: {t_nodot:.4f} s | MSE: {mse(y, yhat_nodot):.6f}")
    print(f"Speedup: {t_nodot / t_dot:.1f}x (higher is better)")


if __name__ == "__main__":
    benchmark()

Vectorized (np.dot)   time: 0.0139 s | MSE: 19.995075
No dot (loops)        time: 12.8448 s | MSE: 19.995075
Speedup: 926.8x (higher is better)
